In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s', datefmt='%H:%M:%S',
                    level=logging.DEBUG)

from Main import *

In [ ]:
data = Data()
cal = Calibration(data)
thr = BinaryThreshold(data)
per = PerspectiveTransform(data)
nextLanes = NextFindLanes(data)
lanes = FindLanes(data)
smooth = ExponentialSmoothing(data, decay=.85)
curve = Curvature(data)
off = LaneOffset(data)
decorate = DecorateWithData(data)

In [ ]:
for f in glob.glob('./camera_cal/*.jpg'):
    img = cv2.imread(f)
    cal.show(img)
    break

In [ ]:
for f in glob.glob('test_images/*.jpg'):
    img = cv2.imread(f)
    thr.show(img)
    break

In [ ]:
for f in glob.glob('test_images/*.jpg'):
    img = cv2.imread(f)
    per.show(img)
    break

In [ ]:
for f in glob.glob('test_images/*.jpg'):
    img = cv2.imread(f)
    img = thr.process(img)
    per.show(img)
    break

In [ ]:
for f in glob.glob('test_images/*.jpg'):
    data.rawImg = cv2.imread(f)
    data.undistImg = cal.process(data.rawImg)
    #data.binaryImg = thr.show(data.undistImg)
    data.binaryImg = thr.process(data.undistImg)
    per.show(data.undistImg)
    data.warpedImg = per.process(data.binaryImg)
    lanes.process(data.warpedImg)
    nextLanes.process(data.warpedImg)
    smooth.process(data.warpedImg)
    curve.process(data.warpedImg)
    off.process(data.warpedImg)
    decorate.show(data.undistImg)
    break

In [ ]:
def processor(img):
    data.rawImg = img
    data.undistImg = cal.process(data.rawImg)
    data.binaryImg = thr.process(data.undistImg)
    data.warpedImg = per.process(data.binaryImg)
    if data.curverad is None:
        lanes.process(data.warpedImg)
    nextLanes.process(data.warpedImg)
    smooth.process(data.warpedImg)
    curve.process(data.warpedImg)
    off.process(data.warpedImg)
    return decorate.process(data.undistImg)

#file = 'challenge_video.mp4'
#file = 'harder_challenge_video.mp4'
file = 'project_video.mp4'
clip = VideoFileClip(file)
#clip = clip.subclip(0,5)
processed = clip.fl_image(processor)
processed.write_videofile(file+'.processed.mp4', audio=False)